# Summary
Inputs: Standard activity data directory (uncleaned -> cleaned) + C algorithm output
Outputs: Plot of raw chest data with peaks and valleys from C algorithm and PP algorithm

In [110]:
# Imports
import os
import pandas as pd
import numpy as np
import plotly.io as pio
pio.renderers.default = "jupyterlab"
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from interfaces.postprocessing import BR_rVE_RTformat_wrapper_peak_detection as pp_peak_detection
import analysis.data_importing as imp  # Custom importing module
import analysis.plotting as pl  # Custom plotting module
import interfaces.postprocessing as pif  # post-processing interface
import scipy.signal
import warnings

import external.custom_post.custom_post_copy as cpc

warnings.simplefilter(action='ignore', category=FutureWarning)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
# Data Cleaning Block
activity_data_dir = "data/c220e0e4-d30d-4ead-858a-1545b91bc362"
uncleaned_data_dir = os.path.join(activity_data_dir, "uncleaned_data")
cleaned_data_dir = os.path.join(activity_data_dir, "cleaned_data")
imp.clean_all_data(uncleaned_data_dir)
clean_dfs = imp.load_cleaned_data(cleaned_data_dir)
raw_slow_df = clean_dfs["raw_slow_df"]
pp_df = clean_dfs["aws_b3_df"]

Created cleaned_data folder at data/c220e0e4-d30d-4ead-858a-1545b91bc362\cleaned_data


In [112]:
# Get outputs from C algorithm
c_df = pd.read_csv(os.path.join(activity_data_dir, "Offline_C_2023-04-25_Normal_Activity_Raw.csv"))
c_df = c_df.loc[:1007,:] # Brian_Chemel's offline C algorithm output contains extra appended rows


In [113]:
# Subtract constant delay from timesteps
filter_length = 10
sample_frequency = 25
delay = filter_length / (2 * sample_frequency)

# Subtract delay from C algorithm output, ensure time is not negative
c_df["PeakTS"] = (c_df["PeakTS"] - delay).clip(lower = 0).round(2)
c_df["ValleyTS"] = (c_df["ValleyTS"] - delay).clip(lower = 0).round(2)


In [142]:
# Get raw chest data
raw_chest = pd.DataFrame(raw_slow_df["c"]).set_index(raw_slow_df["time"])

In [115]:
# Get value of raw chest at timestamp of c_df["ValleyTS"]
c_df["ValleyVal"] = raw_chest.loc[c_df["ValleyTS"]]["c"].values
# Get value of raw chest at timestamp of c_df["PeakTS"]
c_df["PeakVal"] = raw_chest.loc[c_df["PeakTS"]]["c"].values

In [116]:
# For plotting purposes, create a dictionary of colors for each algorithm
color_dict = {
    "raw": "#1f77b4",
    "pp": "#d62728",
    "c": "#2ca02c",
    "err": "#ff7f0e"

}

In [143]:
# Create raw chest trace
raw_chest_trace = go.Scatter(
    x = raw_chest.index,
    y = raw_chest["c"],
    mode = "lines",
    name = "Raw Chest",
    legendgroup= 1,
    line = {"color" : color_dict["raw"]}
)

In [118]:
# Create peak traces (markers - no lines) from C algorithm
peak_val_c_trace = go.Scatter(
    x = c_df["PeakTS"],
    y = c_df["PeakVal"],
    mode = "markers",
    marker_symbol = "triangle-up",
    marker_size = 15,
    marker_color = color_dict["c"],
    marker_line_width = 2,
    marker_line_color = "black",
    legendgroup= 1,
    name = "Peak Val (C)",
    customdata= c_df.index,
    hovertemplate = "Peak Val (C): %{y} <br> Peak Time: %{x} <br> Index: %{customdata}"
)
# add the index of the peak to hover info

# fig2 = go.Figure()
# fig2.add_trace(raw_chest_trace)
# fig2.add_trace(peak_val_c_trace)
# fig2.show(renderer = "browser")

In [119]:
# Create valley traces (markers - no lines) from C algorithm
valley_val_c_trace = go.Scatter(
    x = c_df["ValleyTS"],
    y = c_df["ValleyVal"],
    mode = "markers",
    marker_symbol = "triangle-down",
    marker_size = 15,
    marker_color = color_dict["c"],
    marker_line_width = 2,
    marker_line_color = "black",
    legendgroup= 1,
    name = "Valley Val (C)",
    customdata= c_df.index,
    hovertemplate = "Valley Val (C): %{y} <br> Valley Time: %{x} <br> Index: %{customdata}"
)


In [140]:
## Run post processing peak detection algorithm on raw chest data
pp_pd_df = pp_peak_detection(uncleaned_data_dir) # pp_peak_detection is a wrapper function that calls actual functions within pp algorithm

In [144]:
# Get peak and valley values from raw
pp_pd_df["PeakVal"] = raw_chest.loc[pp_pd_df["PeakTS"]]["c"].values
pp_pd_df["ValleyVal"] = raw_chest.loc[pp_pd_df["ValleyTS"]]["c"].values

In [145]:
# Create peak traces (markers - no lines) from PP algorithm
peak_val_pp_trace = go.Scatter(
    x = pp_pd_df["PeakTS"],
    y = pp_pd_df["PeakVal"],
    mode = "markers",
    marker_symbol = "triangle-up",
    marker_size = 15,
    marker_color = color_dict["pp"],
    marker_line_width = 2,
    marker_line_color = "black",
    name = "Peak Val (PP)",
    legendgroup= 1,
    customdata= pp_pd_df.index,
    hovertemplate = "Peak Val (PP): %{y} <br> Peak Time: %{x} <br> Index: %{customdata}"
)

In [146]:
# Create valley traces (markers - no lines) from PP algorithm
valley_val_pp_trace = go.Scatter(
    x = pp_pd_df["ValleyTS"],
    y = pp_pd_df["ValleyVal"],
    mode = "markers",
    marker_symbol = "triangle-down",
    marker_size = 15,
    marker_color = color_dict["pp"],
    marker_line_width = 2,
    marker_line_color = "black",
    legendgroup= 1,
    name = "Valley Val (PP)",
    customdata= pp_pd_df.index,
    hovertemplate = "Valley Val (PP): %{y} <br> Valley Time: %{x} <br> Index: %{customdata}"
)


In [147]:
# Create the post-processing VT trace
VT_pp_trace = go.Scatter(
    x = pp_df["breathTime"],
    y = pp_df["VT"],
    mode = "lines",
    name = "VT (PP)",
    line = {"color" : color_dict["pp"]},
    legendgroup = 2
)
# Create the C VT trace
VT_c_trace = go.Scatter(
    x = c_df["PeakTS"],
    y = c_df["VT"],
    mode = "lines",
    name = "VT (C)",
    line = {"color" : color_dict["c"]},
    legendgroup = 2
)
# Plot VT traces
# fig2 = go.Figure()
# fig2.add_trace(VT_pp_trace)
# fig2.add_trace(VT_c_trace)
# fig2.show(renderer = "browser")

In [149]:
# Make the Peak detection plot collection
# make subplots
fig3 = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)
# Add title
fig3.update_layout(title_text="Peak Detection (Top) and VT (Bottom)")
fig3.add_trace(raw_chest_trace, row=1, col=1, )
fig3.add_trace(raw_chest_trace, row=2, col=1, )
fig3.add_trace(peak_val_c_trace, row=2, col=1)
fig3.add_trace(valley_val_c_trace, row=2, col=1)
fig3.add_trace(peak_val_pp_trace, row=2, col=1)
fig3.add_trace(valley_val_pp_trace, row=2, col=1)
# add subplot title
fig3.update_yaxes(title_text="Raw Chest Magnitude", row=1, col=1)

fig3.add_trace(VT_c_trace, row=3, col=1)
fig3.add_trace(VT_pp_trace, row=3, col=1)
fig3.update_yaxes(title_text="VT", row=3, col=1)
fig3.update_xaxes(title_text="Time[s]", row=3, col=1)

fig3.update_layout(height=500*2,
                    legend_tracegroupgap=450,
                    )


fig3.show(renderer = "browser")
pl.figures_to_html([fig3], "PeakDetection_VT.html", show = False)

## Identifying Error Regions
There are differences in the detected peaks between the two algorithms, but what is the impact on the Minute Volume calculuation


In [126]:
# Plot VE from both algorithms

## C VE
VE_c_trace = go.Scatter(
    x = c_df["PeakTS"],
    y = c_df["VE"],
    mode = "lines",
    name = "VE (C)",
    line = {"color" : color_dict["c"]},
)
## PP VE
VE_pp_trace = go.Scatter(
    x = pp_df["breathTime"],
    y = pp_df["VE"],
    mode = "lines",
    name = "VE (PP)",
    line = {"color" : color_dict["pp"]},
)
# Plot VE traces
# fig4 = go.Figure()
# fig4.add_trace(VE_c_trace)
# fig4.add_trace(VE_pp_trace)
# fig4.show(renderer = "browser")


## Get error signal between VE's

In [127]:
## First interpolate each time-series to the same time base
VE_pp = pd.DataFrame(pp_df["VE"]).set_axis(pp_df["breathTime"])
VE_c = pd.DataFrame(c_df["VE"]).set_axis(c_df["PeakTS"])
VE = VE_pp.join(VE_c, how="outer", rsuffix="_c", lsuffix="_pp")
VE_interp = VE.interpolate(method="index")
d = pd.DataFrame(np.arange(max(VE_interp.index)), index=np.arange(max(VE_interp.index)))
VE_sec = VE_interp.join(d, how="outer")
# VT_j_int_sec # Uncomment to view
VE_sec_int = VE_sec.interpolate(method="index").fillna(value=0).drop(columns=[0])

VE_c_sec_trace = go.Scatter(
    x = VE_sec_int.index,
    y = VE_sec_int["VE_c"],
    mode = "lines",
    name = "VE (C)",
    line = {"color" : color_dict["c"]},
)

VE_pp_sec_trace = go.Scatter(
    x = VE_sec_int.index,
    y = VE_sec_int["VE_pp"],
    mode = "lines",
    name = "VE (PP)",
    line = {"color" : color_dict["pp"]},
)

# Plot VE and VE_sec traces
# fig5 = go.Figure()
# fig5.add_trace(VE_c_trace)
# fig5.add_trace(VE_pp_trace)
# fig5.add_trace(VE_c_sec_trace)
# fig5.add_trace(VE_pp_sec_trace)
# fig5.show(renderer = "browser")

In [128]:
## Compute error in the VE Sec signals
VE_sec_int["VE_err"] = (VE_sec_int["VE_c"] - VE_sec_int["VE_pp"])/VE_sec_int["VE_pp"]
VE_err_trace = go.Scatter(
    x = VE_sec_int.index,
    y = VE_sec_int["VE_err"],
    mode = "lines",
    name = "VE Error",
    line = {"color" : color_dict["err"]},
)


In [129]:
# Create plot collection of VE and Errors
fig6 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                     subplot_titles=("VE (Top) and VE Error (Bottom)"))
# Add title
fig6.update_layout(title_text="VE (Top) and VE Error (Bottom)")
fig6.add_trace(VE_c_sec_trace, row=1, col=1, )
fig6.add_trace(VE_pp_sec_trace, row=1, col=1)
fig6.add_trace(VE_err_trace, row=2, col=1)
# add subplot title
fig6.update_yaxes(title_text="VE", row=1, col=1)
fig6.update_yaxes(title_text="VE Error", row=2, col=1)
fig6.update_xaxes(title_text="Time[s]", row=2, col=1)
# space the legends
fig6.update_layout(height=500*2,
                    legend_tracegroupgap=450,
                    )

# show fig6
fig6.show(renderer = "browser")

In [130]:
## Shade the back of the VE plot according to the error
# Create a new trace for the error region
VE_err_region_trace = go.Scatter(
    x = VE_sec_int.index,
    y = VE_sec_int["VE_c"],
    mode = "lines",
    name = "VE (C)",
    line = {"color" : color_dict["c"]},
    fill = "tonexty",
    fillcolor = color_dict["err"],
)
# Create a new trace for the error region
VE_err_region_trace2 = go.Scatter(
    x = VE_sec_int.index,
    y = VE_sec_int["VE_pp"],
    mode = "lines",
    name = "VE (PP)",
    line = {"color" : color_dict["pp"]},
    fill = "tonexty",
    fillcolor = color_dict["err"],
)

In [131]:
if False:
    # Create plot collection of VE andErrors
    fig7 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
    # Add title
    fig7.update_layout(title_text="VE (Top) and VE Error (Bottom)")
    fig7.add_trace(VE_c_sec_trace, row=1, col=1, )
    fig7.add_trace(VE_pp_sec_trace, row=1, col=1)
    fig7.add_trace(VE_err_trace, row=2, col=1)
    fig7.add_trace(VE_err_region_trace, row=1, col=1)
    fig7.add_trace(VE_err_region_trace2, row=1, col=1)
    # add subplot title
    fig7.update_yaxes(title_text="VE", row=1, col=1)
    fig7.update_yaxes(title_text="VE Error", row=2, col=1)
    fig7.update_xaxes(title_text="Time[s]", row=2, col=1)
    # space the legends
    fig7.update_layout(height=500*2,
                        legend_tracegroupgap=450,
                        )

    # show fig7
    fig7.show(renderer = "browser")


# Peak Detection and VE Error

In [132]:
# Make the Peak detection plot collection
# make subplots
fig8 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                     subplot_titles=["Raw chest with peak detection", "VE Error"])
# Add title
fig8.update_layout(title_text="Peak Detection vs VE Error")
fig8.add_trace(raw_chest_trace, row=1, col=1, )
fig8.add_trace(peak_val_c_trace, row=1, col=1)
fig8.add_trace(valley_val_c_trace, row=1, col=1)
fig8.add_trace(peak_val_pp_trace, row=1, col=1)
fig8.add_trace(valley_val_pp_trace, row=1, col=1)
# add subplot title
fig8.update_yaxes(title_text="Raw Chest Magnitude", row=1, col=1)

fig8.add_trace(VE_err_trace, row=2, col=1)
fig8.update_yaxes(title_text="VE Error", row=2, col=1)
fig8.update_xaxes(title_text="Time[s]", row=2, col=1)

fig8.update_layout(height=500*2,
                    legend_tracegroupgap=450,
                    )


fig8.show(renderer = "browser")


# Breathing Rate Comparison

In [133]:
# Plot instant breathing rate and average breathing rate

instBR_c_trace = go.Scatter(
    x = c_df["PeakTS"],
    y = c_df["InstBR"],
    mode = "lines",
    name = "Instant BR (C)",
    line = {"color" : color_dict["c"]},
)
AvgBr_c_trace = go.Scatter(
    x = c_df["PeakTS"],
    y = c_df["AvgBR"],
    mode = "lines",
    name = "Avg BR (C)",
    line = {"color" : color_dict["c"]},
)

instBR_pp_trace = go.Scatter(
    x = pp_df["breathTime"],
    y = pp_df["instBR"],
    mode = "lines",
    name = "Instant BR (PP)",
    line = {"color" : color_dict["pp"]},
)

RRAvg_pp_trace = go.Scatter(
    x = pp_df["breathTime"],
    y = pp_df["RRAvg"],
    mode = "lines",
    name = "RR Avg (PP)",
    line = {"color" : color_dict["pp"]},
)

# plot traces
fig9 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, subplot_titles=("Instant BR", "Avg BR"))
# Add title
fig9.update_layout(title_text="Breathing Rate Comparison")
fig9.add_trace(instBR_c_trace, row=1, col=1, )
fig9.add_trace(instBR_pp_trace, row=1, col=1)
fig9.add_trace(AvgBr_c_trace, row=2, col=1)
fig9.add_trace(RRAvg_pp_trace, row=2, col=1)

# add subplot title
fig9.update_yaxes(title_text="Instant BR", row=1, col=1)
fig9.update_yaxes(title_text="Avg BR", row=2, col=1)

fig9.update_xaxes(title_text="Time[s]", row=2, col=1)

fig9.update_layout(height=500*2,
                    legend_tracegroupgap=450,
                    )

fig9.show(renderer = "browser")


In [134]:
# Plot the intant and average breathing rate together
fig10 = make_subplots(rows = 2, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                      subplot_titles=("C Algorithm", "PP Algorithm"))
fig10.update_layout(title_text="Instant vs Average Breathing Rate")

AvgBr_c_trace_dash = go.Scatter(
    x = c_df["PeakTS"],
    y = c_df["AvgBR"],
    mode = "lines",
    name = "Avg BR (C)",
    line = {"color" : color_dict["c"], "dash" : "dash"},
)
RRAvg_pp_trace_dash = go.Scatter(
    x = pp_df["breathTime"],
    y = pp_df["RRAvg"],
    mode = "lines",
    name = "RR Avg (PP)",
    line = {"color" : color_dict["pp"], "dash" : "dash"},
)
fig10.add_trace(instBR_c_trace, row=1, col=1)
fig10.add_trace(AvgBr_c_trace_dash, row=1, col=1)
fig10.add_trace(instBR_pp_trace, row=2, col=1)
fig10.add_trace(RRAvg_pp_trace_dash, row=2, col=1)

fig10.update_yaxes(title_text="C BR", row=1, col=1)
fig10.update_yaxes(title_text="PP BR", row=2, col=1)

fig10.update_xaxes(title_text="Time[s]", row=2, col=1)

fig10.show(renderer = "browser")


In [135]:
# Alignment of metrics based on cross-correlation
def cross_correlate(series1, series2):
    sig1 = series1.dropna()
    sig2 = series2.dropna()
    corr = scipy.signal.correlate(sig1, sig2)
    lags = scipy.signal.correlation_lags(len(sig1), len(sig2))

    return corr / corr.max(), lags


def cross_corr_align(df1, df2, ycol1, ycol2, xcol1, xcol2):
    """
    This function takes in two dataframes and aligns them based on the cross-correlation
    between two columns. The function returns the lag value and the aligned dataframe.
    """
    from copy import deepcopy
    # First we need to join and interpolate the two series
    df1 = pd.DataFrame(df1[ycol1].set_axis(df1[xcol1]))
    df2 = pd.DataFrame(df2[ycol2].set_axis(df2[xcol2]))
    df = df1.join(df2, how="outer", rsuffix="_1", lsuffix="_2")
    df_interp = df.interpolate(method="index")
    sec_df = pd.DataFrame(np.arange(max(df_interp.index)), index=np.arange(max(df_interp.index)))
    df_sec = df_interp.join(sec_df, how="outer")
    df_sec = df_sec.interpolate(method="index").fillna(value =0).drop(columns=[0])

    # Now we can compute the cross correlation
    ycol_1 = ycol1 + "_1"
    ycol_2 = ycol2 + "_2"
    corr, lags = cross_correlate(df_sec[ycol_1], df_sec[ycol_2])
    opt_lag = lags[np.argmax(corr)]

    # Copy df sec
    df_sec_shift = deepcopy(df_sec)
    # Shift the second column by the optimal lag
    df_sec_shift[df_sec.columns[0]] = df_sec_shift[df_sec.columns[0]].shift(opt_lag)

    return df_sec_shift, opt_lag, df_sec


In [136]:
if False:
    # Test cross_corr_align function
    df_sec_shift, opt_lag, df_sec = cross_corr_align(c_df, pp_df, "VT", "VT", "PeakTS", "breathTime")

    color_map = {df_sec.columns[0]: "lightseagreen", df_sec.columns[1]: "darkorange"}

    # plot df_sec_shift and df_sec
    fig11 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
    # Add title
    fig11.update_layout(title_text=f"Shifted - {opt_lag}  (Top), Unshifted (Bottom)")
    fig11.add_trace(go.Scatter(
        x = df_sec_shift.index,
        y = df_sec_shift[df_sec_shift.columns[0]],
        mode = "lines",
        name = df_sec_shift.columns[0],
        line={"color": color_map[df_sec_shift.columns[0]]},
    ), row=1, col=1)

    fig11.add_trace(go.Scatter(
        x = df_sec_shift.index,
        y = df_sec_shift[df_sec_shift.columns[1]],
        mode = "lines",
        name = df_sec_shift.columns[1],
        line={"color": color_map[df_sec_shift.columns[1]]},
    ), row=1, col=1)

    fig11.add_trace(go.Scatter(
        x = df_sec.index,
        y = df_sec[df_sec.columns[0]],
        mode = "lines",
        name = df_sec.columns[0],
        line={"color": color_map[df_sec.columns[0]]},
    ), row=2, col=1)

    fig11.add_trace(go.Scatter(
        x = df_sec.index,
        y = df_sec[df_sec.columns[1]],
        mode = "lines",
        name = df_sec.columns[1],
        line={"color": color_map[df_sec.columns[1]]},
    ), row=2, col=1)

    fig11.show(renderer = "browser")
